## FINAL TASK

In short, your task is to find out all time warmest and coldest months from the data, and create a plot that shows the all-time warmest and coldest mean monthly temperatures from January to December:

![img/monthly_min_max.png](img/monthly_min_max.png)

### Input data

Use one of the data files from lesson 6. If you want, you can of course repeate this process for all the weather stations! 

### Steps

You can design the analysis workflow freely, but in general you will need to include these steps in your analysis:

1. Read in data and parse the dates
2. Convert Fahreheit to Celsius
3. Calculate monthly mean temperatures for each unique month in the data (you can subset the data to a 30 year observation period)
4. Calculate all time maximum and minimum temperatures for each month (January-December) based on the monthly means
5. Plot all time minimum and maximum temperatures in one plot (months on the x-axis, temperature on the y-axis)

Extra: plot an interactive plot (using Bokeh) where you can hover over the values and see which year did the minimum and maximum temperatures occur.

In [ ]:
import pandas as pd

# remember to comment your code using inline comments and markdown cells :)